In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext wurlitzer

In [3]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level = logging.DEBUG)

In [4]:
import pyfdb
import findlibs
import yaml
from pathlib import Path
import os
import pandas as pd
import pyodc
import shutil

In [5]:
from pathlib import Path
from ionbeam.core.config_parser import parse_config
from ionbeam.core.bases import *
from ionbeam.core.source import Source
from IPython.display import display
from ionbeam.metadata.db import init_db
import sqlalchemy
from datetime import datetime, UTC, timedelta




config_file = Path("~/git/IonBeam_bundle/IonBeam/config/").expanduser()
config, actions = parse_config(config_file,
                    environment  = "local",
                    sources = ["smart_citizen_kit"],
                    download = True,
                    die_on_error = True,
                    reingest_from = datetime.fromisoformat("2021-01-01").replace(tzinfo=UTC),
                    )

config.globals.ingestion_time_constants.query_timespan = dataclasses.replace(
    config.globals.ingestion_time_constants.query_timespan,
    start = datetime.now(UTC) - timedelta(hours = 10))

# init_db(config.globals) 

DEBUG:ionbeam.core.config_parser.config_parser:Configuration Directory: /Users/math/git/IonBeam_bundle/IonBeam/config
DEBUG:ionbeam.core.config_parser.config_parser:Global config file: /Users/math/git/IonBeam_bundle/IonBeam/config/config.yaml
DEBUG:ionbeam.core.config_parser.config_parser:Loaded global config...
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.data_path to /Users/math/git/IonBeam_bundle/data
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.cache_path to /Users/math/git/IonBeam_bundle/data/cache
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_schema_path to /Users/math/git/IonBeam_bundle/IonBeam/config/schema
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.fdb_root to /Users/math/git/IonBeam_bundle/data/fdb_root
DEBUG:ionbeam.core.config_parser.config_parser:Resolved config.globals.metkit_language_template to /Users/math/git/IonBeam_bundle/IonBeam/config/language.yaml.template


In [6]:
from uuid import UUID

id2action = {a.id : a for a in actions}
sources = [a for a in actions if isinstance(a, Source)]
processors = [a for a in actions if not isinstance(a, Source)]

# chains = [[s,] for s in sources]
# for a in processors:
#     if not isinstance(a.match, UUID): 
#         chains.append([str(a.match), a,])
#         continue
#     for c in chains:
#         matches = a.match.int == c[-1].id.int
#         if matches: c.append(a)
            

# for c in chains:
#     print(" --> ".join(str(a) for a in c))

In [7]:
sources, stateless_actions, aggregators = [], [], []
for action in actions:
    if isinstance(action, Source):
        sources.append(action)
    elif isinstance(action, Aggregator):
        aggregators.append(action)
    else:
        stateless_actions.append(action)

print("Sources")
for i, a in enumerate(sources):
    display(a)

print("Aggregators")
for i, a in enumerate(aggregators):
    display(a)

print("Actions")
for i, a in enumerate(stateless_actions):
    display(a)

Sources


name,None
forward_to_names,None
id,673851a9-3b23-4dc5-8408-22e1e411794d
next,[SetConstants(id = ...c0b79)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
copy_metadata_to_columns,"{'external_station_id': 'device.id', 'station_name': 'device.name', 'lat': 'device.location.latitude', 'lon': 'device.location.longitude', 'author': 'device.owner.username'}"
finish_after,None
cache_version,3
use_cache,True
source,smart_citizen_kit
maximum_request_size,"1 day, 0:00:00"


Aggregators
Actions


name,None
forward_to_names,None
id,9a24318f-4cbe-48b2-b171-020c210c0b79
next,[ComputeStationId(id = ...493b9)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,673851a9-3b23-4dc5-8408-22e1e411794d
set,"{'platform': 'smart_citizen_kit', 'aggregation_type': 'chunked'}"
name,value
source_action_id,9a24318f-4cbe-48b2-b171-020c210c0b79
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation, e.g. acronet, meteotracker, smart_citizen_kit'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. chunked or whole'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), CanonicalVariable(name='start_time', unit=None, desc='For chunked data files, the logical begining of the time period of the chunk, possibly rounded.'), CanonicalVariable(name='offset_tz', unit=None, desc='Uncertain what this means in the Meteotracker data.'), CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVar..."


name,None
forward_to_names,None
id,a4a8b586-c434-4c39-9743-999e036493b9
next,[ComputeChunkDateTime(id = ...a26d1)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,9a24318f-4cbe-48b2-b171-020c210c0b79
name,value
source_action_id,a4a8b586-c434-4c39-9743-999e036493b9
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation, e.g. acronet, meteotracker, smart_citizen_kit'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. chunked or whole'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), CanonicalVariable(name='start_time', unit=None, desc='For chunked data files, the logical begining of the time period of the chunk, possibly rounded.'), CanonicalVariable(name='offset_tz', unit=None, desc='Uncertain what this means in the Meteotracker data.'), CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVar..."
data_path,/Users/math/git/IonBeam_bundle/data


...

name,None
forward_to_names,None
id,aed314e8-11aa-483d-b15f-f12b9ba5db19
next,[ComputeMARSIdentifier(id = ...a4d54)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,73eda57e-801f-4ee2-94ac-4193142dda7e
name,value
source_action_id,aed314e8-11aa-483d-b15f-f12b9ba5db19
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation, e.g. acronet, meteotracker, smart_citizen_kit'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. chunked or whole'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), CanonicalVariable(name='start_time', unit=None, desc='For chunked data files, the logical begining of the time period of the chunk, possibly rounded.'), CanonicalVariable(name='offset_tz', unit=None, desc='Uncertain what this means in the Meteotracker data.'), CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVar..."
data_path,/Users/math/git/IonBeam_bundle/data


name,None
forward_to_names,[final_processing]
id,acad9173-3310-425f-a481-16c7540a4d54
next,[IdentityAction(id = ...97cfa)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/smart_citizen_kit/actions.yaml
match,aed314e8-11aa-483d-b15f-f12b9ba5db19
lookup,"{'class': 'const.rd', 'expver': 'const.xxxx', 'stream': 'const.lwda', 'aggregation_type': 'data.aggregation_type', 'platform': 'data.platform', 'date': 'data.chunk_date', 'internal_id': 'data.station_id', 'time': 'data.chunk_time', 'version': 'const.1'}"
name,value
source_action_id,acad9173-3310-425f-a481-16c7540a4d54
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation, e.g. acronet, meteotracker, smart_citizen_kit'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. chunked or whole'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), CanonicalVariable(name='start_time', unit=None, desc='For chunked data files, the logical begining of the time period of the chunk, possibly rounded.'), CanonicalVariable(name='offset_tz', unit=None, desc='Uncertain what this means in the Meteotracker data.'), CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVar..."


name,final_processing
forward_to_names,None
id,820db434-5117-47e4-b49e-3207a9697cfa
next,[ParquetEncoder(id = ...a86a5)]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,None
name,value
source_action_id,820db434-5117-47e4-b49e-3207a9697cfa
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation, e.g. acronet, meteotracker, smart_citizen_kit'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. chunked or whole'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), CanonicalVariable(name='start_time', unit=None, desc='For chunked data files, the logical begining of the time period of the chunk, possibly rounded.'), CanonicalVariable(name='offset_tz', unit=None, desc='Uncertain what this means in the Meteotracker data.'), CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVar..."
data_path,/Users/math/git/IonBeam_bundle/data


name,None
forward_to_names,None
id,01fb7409-1378-4a1b-ac50-d249230a86a5
next,[FDBWriter()]
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,820db434-5117-47e4-b49e-3207a9697cfa
name,value
source_action_id,01fb7409-1378-4a1b-ac50-d249230a86a5
state,encoded
name,value
canonical_variables,"[CanonicalVariable(name='platform', unit=None, desc='The platform that made the observation, e.g. acronet, meteotracker, smart_citizen_kit'), CanonicalVariable(name='aggregation_type', unit=None, desc='The type of aggregation of the data, e.g. chunked or whole'), CanonicalVariable(name='sensor_name', unit=None, desc='The name of the sensor that made the observation.'), CanonicalVariable(name='datetime', unit=None, desc='The date and time that the observation was made.'), CanonicalVariable(name='start_time', unit=None, desc='For chunked data files, the logical begining of the time period of the chunk, possibly rounded.'), CanonicalVariable(name='offset_tz', unit=None, desc='Uncertain what this means in the Meteotracker data.'), CanonicalVariable(name='lat', unit='EPSG:4326', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVariable(name='lon', unit='EPSG:4326', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)'), CanonicalVar..."


name,None
forward_to_names,None
id,bc480e95-f5b7-42ad-892d-22e474c3693e
next,None
definition_path,/Users/math/git/IonBeam_bundle/IonBeam/config/actions.yaml
match,01fb7409-1378-4a1b-ac50-d249230a86a5
config,"{'engine': 'toc', 'spaces': [{'handler': 'Default', 'roots': [{'path': '/Users/math/git/IonBeam_bundle/data/fdb_root'}]}], 'schema': '/Users/math/git/IonBeam_bundle/IonBeam/config/schema'}"
debug,[]
name,value
source_action_id,bc480e95-f5b7-42ad-892d-22e474c3693e
state,written


In [8]:
from datetime import datetime, timedelta

online = True
source = sources[0]
source.use_cache = True
incoming_message_stream = source.generate()
input_message = next(incoming_message_stream)



DEBUG:ionbeam.sources.smart_citizen_kit.source:Tag 'Barcelona' has 25 devices
DEBUG:ionbeam.sources.smart_citizen_kit.source:Tag 'I-CHANGE' has 25 devices
DEBUG:ionbeam.sources.smart_citizen_kit.source:User 'ichangecima' has 14
DEBUG:ionbeam.sources.smart_citizen_kit.source:User 'ichange_project' has 1
DEBUG:ionbeam.sources.smart_citizen_kit.source:Found 65 devices overall for I-CHANGE.
DEBUG:ionbeam.sources.smart_citizen_kit.source:4 of those might have data in the requested date range.
INFO:ionbeam.sources.API_sources_base:For source smart_citizen_kit, got data streams [DataStream(smart_citizen_kit, sck:id=16517, 1, None), DataStream(smart_citizen_kit, sck:id=16643, 1, None), DataStream(smart_citizen_kit, sck:id=16774, 1, None), DataStream(smart_citizen_kit, sck:id=16030, 1, None)]
INFO:ionbeam.sources.API_sources_base:New or updated chunks 12
INFO:ionbeam.sources.API_sources_base:Affected time spans 11
DEBUG:ionbeam.sources.API_sources_base:Starting to emit messages, looping over ti

ValueError: Indexes have overlapping values: MultiIndex([('2025-01-16 16:52:54+00:00', 'smart_citizen_kit/device/16030'),
            ('2025-01-16 17:00:54+00:00', 'smart_citizen_kit/device/16030'),
            ('2025-01-16 16:52:37+00:00', 'smart_citizen_kit/device/16774'),
            ('2025-01-16 17:00:46+00:00', 'smart_citizen_kit/device/16517'),
            ('2025-01-16 17:00:37+00:00', 'smart_citizen_kit/device/16774'),
            ('2025-01-16 17:00:48+00:00', 'smart_citizen_kit/device/16643'),
            ('2025-01-16 16:52:48+00:00', 'smart_citizen_kit/device/16643'),
            ('2025-01-16 16:52:46+00:00', 'smart_citizen_kit/device/16517')],
           names=['datetime', 'external_station_id'])

In [ ]:
time_span = TimeSpan.last(days = 50)

datastreams = list(source.get_data_streams(time_span))
datastreams

In [ ]:
# with Session(config.globals.sql_engine) as db_session:
#     for ds in datastreams:
#         existing_chunks = ds.get_chunks(db_session, time_span)
#         print(list(existing_chunks))
    

In [ ]:
# from matplotlib import pyplot as plt
# from datetime import UTC

# show_time_span = TimeSpan.last(days = 100)

# def plot_datastreams(data_streams):

#     n = len(data_streams)
#     f, axes = plt.subplots(n, 1, figsize = (10, 2*n), sharex = True, squeeze = False)
    
#     def plot_timespan(ax, ts, colour, label = None):
#         ax.axvline(ts.start, linestyle = "dotted", color = colour)
#         ax.axvline(ts.end, linestyle = "solid", color = colour)
#         ax.axvspan(ts.start, ts.end, alpha = 0.1, color = colour, label = label)
    
#     with Session(config.globals.sql_engine) as db_session:
#         for ds, ax in zip(data_streams, axes[0]):
#             existing_chunks = ds.get_chunks(db_session, show_time_span)
            
#             for i, chunk in enumerate(existing_chunks):
#                 if not chunk.success:
#                     pprint(chunk)
#                 colour = "red" if not chunk.success else "green"
#                 label = "Failed" if not chunk.success else "Sucess"
#                 plot_timespan(ax, chunk.time_span, colour, label = label)

#                 times = set(datetime.strptime(t, "%Y%m%d%H%M")
#                             for readings in chunk.json
#                             for t in readings["timeline"])
#                 ax.scatter(y = [i%2 for _ in times],  x = list(times), color = "blue")
            
#             gaps = source.gaps_in_database(db_session, ds, show_time_span)
#             for ts in gaps: 
#                 # print(f"gap: {ts.delta()}")
#                 plot_timespan(ax, ts, "blue", label = "Todo")
    
#             ax.axvline(datetime.now(UTC), linestyle = "dotted", color = "k")
#             ax.set(ylabel = ds.key)
#     # ax.legend()

# for source in sources:
#     print(f"Source: {source}")
#     plot_datastreams(list(source.get_data_streams(show_time_span)))

In [ ]:
from copy import deepcopy
from ionbeam.core.bases import FinishMessage, FileMessage, MetaData
from ionbeam.aggregators import TimeAggregator
from ionbeam.core.singleprocess_pipeline import process_message

possible_actions = [a for a in actions if not isinstance(a, Source)]
message = deepcopy(input_message)

message_history = []
actions_run = set()
while True:
    display(message)
    message_history.append(message)
    matching = list(message.find_matches(config.globals.actions_by_id))
    
    if not matching: 
        print("No more matches")
        break

    print("That matched with: ", [str(a) for a in matching])
    action = matching[0]
    if action.id in actions_run:
        print(f"Error: loop found for {action} {action.id = }")
        break

    # print(f"current message id {id(message)}")
    message = next(process_message(message, action))
    actions_run.add(action.id)

    
        

In [ ]:
message_history[0].data

## Reload and retry failed action

In [ ]:
from ionbeam.core.config_parser import reload_action
from IPython.display import display

print(f"Failed action: {action} Definition: {action.definition_path}")

config, action = reload_action(config, action)

message = next(action.process(message_history[-1]))

display(action)
display(message)

In [ ]:
m = message_history[-1]
m.metadata.unstructured

In [ ]:
import pyodc as pyodc
from io import BytesIO

f = BytesIO(message.bytes)
df = pyodc.read_odb(f, single = True)
df

In [ ]:
from ionbeam.core.converters import unit_conversions

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100)})

In [ ]:
converter = unit_conversions["°C -> K"]


%timeit df.apply({"A" : converter}, raw = True, by_row = False)

In [ ]:
converter = lambda t: t + 273.15
converter2 = lambda t: t * 1000

df = pd.DataFrame({"A" : np.arange(100), "B": np.linspace(0,1,100), "C" : np.arange(100)})

df = df.apply({"A" : converter, "B" : converter2}, raw = True, by_row = False)

df

In [ ]:
converter = lambda 